In [1]:
#Dependencies
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import datetime as datetime
import time
#import re
import numpy as np
from sqlalchemy import create_engine


In [2]:
#create splinter browser
executable_path = {'executable_path':"../chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:

#define url as variable and open with browser
url = "https://dallasites101.com/archive/2020-dallas-restaurant-openings-closings-and-coming-soons"
browser.visit(url)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [4]:
#Grab lists of openings and closings
listings = soup.find_all('ul')

In [5]:
# Save lists of buisness type and status for a nested for loop
list_type = ['Restaurant', 'Restaurant', 'Restaurant', 'Bar', 'Bar', 'Bar', 'Sweets', 'Sweets', 'Sweets', 'Cafe', 'Cafe', 'Cafe']
list_status = ['Open', 'Coming Soon', 'Closed', 'Open', 'Coming Soon', 'Closed', 'Open', 'Coming Soon', 'Closed', 'Open', 'Coming Soon', 'Closed',]


In [6]:
name =[]
neighborhood = []
kind = []
status = []
hyperlink = []

#Loop through each ul, then loop through each li.
for x in range(0, (len(listings)-1)):
    for store in listings[x].find_all('li'):
        temp = store.find('p')
        temptext = temp.text.replace("\xa0", " ")
        temptext = temptext.split(' | ')
        name.append(temptext[0])
        temphood = temptext[1].split(' (')[0].split(' \t')[0]
        neighborhood.append(temphood.rstrip())
        kind.append(list_type[x])
        status.append(list_status[x])
        try:
            templink = temp.find('a').get('href')
            hyperlink.append(templink)
        except AttributeError:
            hyperlink.append(np.nan)


In [34]:
status

['Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Open',
 'Coming Soon',
 'Coming Soon',
 'Coming Soon',
 'Coming Soon',
 'Coming Soon',
 'Coming Soon',
 'Coming Soon

In [7]:
#Load data into dataframe

data = {'Name': name, 'Neighborhood': neighborhood, 'Type': kind, 'Status': status, 'URL': hyperlink}

df = pd.DataFrame(data)

In [8]:
#Clean neighborhoods
df.loc[df['Neighborhood'] == 'Downtown Dallas', 'Neighborhood'] = 'Downtown'
df.loc[df['Neighborhood'] == 'Legacy Food Hall', 'Neighborhood'] = 'Legacy Hall'
df.loc[df['Neighborhood'] == 'Lower Greenville', 'Neighborhood'] = 'Greenville'
df.loc[df['Neighborhood'] == 'Greenville', 'Neighborhood'] = 'Greenville'
df.loc[df['Neighborhood'] == 'Henderson Ave', 'Neighborhood'] = 'Henderson'
df.loc[df['Neighborhood'] == 'Delivery only in Grapevine', 'Neighborhood'] = 'Delivery'
df.loc[df['Neighborhood'] == 'Highland Park Village', 'Neighborhood'] = 'Highland Park'
df.loc[df['Neighborhood'] == 'Ross and Hall', 'Neighborhood'] = 'Ross Ave'
df.loc[df['Neighborhood'] == 'State Thomas', 'Neighborhood'] = 'Uptown'
df.loc[df['Neighborhood'] == 'Location TBD', 'Neighborhood'] = np.nan


In [9]:
df['Neighborhood'].unique()

array(['Trinity Groves', 'Oak Cliff', 'Grapevine', 'Deep Ellum',
       'Henderson', 'Design District', 'Plano', 'Bishop Arts',
       'Arlington', 'Uptown', 'Downtown', 'Legacy Hall', 'Greenville',
       'Northwest Dallas', 'East Dallas', 'Preston Hollow',
       'Greenville Ave', 'Frisco', 'West Dallas', 'McKinney', 'Bluffview',
       'The Hill', 'Lakewood', 'Grand Prairie', 'Carrollton', 'Fitzhugh',
       'Lemmon Ave', 'Oak Lawn', 'South Dallas', 'Irving',
       'Lake Highlands', 'Victory Park', 'SMU', 'Preston Royal',
       'Las Colinas', 'Delivery', 'The Colony', 'North Dallas',
       'Fair Park', 'Richardson', 'DeSoto', 'Addison', 'Northeast Dallas',
       'Arts District', 'Highland Park', 'Casa Linda',
       'Dallas, Plano, Arlington', 'Dallas', 'Park Cities',
       'West Village', 'Greeenville Ave', 'Vickery Meadow',
       'Mockingbird Station', 'Klyde Warren Park',
       'Lemmon Ave and Greenville', 'Harwood', 'Ross Ave',
       'Farmer’s Branch', 'Medical District'

In [10]:
df.sample(5)

,Name,Neighborhood,Type,Status,URL
3,Basic Taco,Deep Ellum,Restaurant,Open,http://basicta.co/
188,Gaslight,Uptown,Bar,Open,https://www.facebook.com/gaslightdtx/
207,Vitruvian Park Tavern,Addison,Bar,Open,https://www.vitruvianparktavern.com
269,Purify Bowls,Deep Ellum,Cafe,Open,NaN
17,Dahlia Bar And Bistro,East Dallas,Restaurant,Open,https://dahliaonross.com/


In [11]:
#create  unique IDs for SQL tables
df['nameID'] = df.groupby(['Name']).ngroup()
df['neighborhoodID'] = df.groupby(['Neighborhood']).ngroup()
df['typeID'] = df.groupby(['Type']).ngroup()
df['statusID'] = df.groupby(['Status']).ngroup()
df['urlID'] = df.groupby(['URL']).ngroup()


In [12]:
#Change urlID -1 into None
df.loc[df.urlID == -1,['urlID']] = np.nan
df.sample(10)

,Name,Neighborhood,Type,Status,URL,nameID,neighborhoodID,typeID,statusID,urlID
117,Meridian,North Dallas,Restaurant,Coming Soon,NaN,147,40,2,1,NaN
162,Paul Martin’s Grill,Oak Lawn,Restaurant,Closed,NaN,170,45,2,0,NaN
29,Hello Dumpling,The Hill,Restaurant,Open,https://hello-dumpling.com/,95,56,2,2,38.0
272,SoCo Coffee,Lake Highlands,Cafe,Open,https://socogoodeats.com/,202,31,1,2,55.0
44,La Resistencia,Deep Ellum,Restaurant,Open,https://revolvertacolounge.com/la-resistencia/,127,10,2,2,51.0
125,Bar Taco,North Dallas,Restaurant,Closed,NaN,11,40,2,0,NaN
158,Neighborhood Services,Preston Royal,Restaurant,Closed,NaN,155,50,2,0,NaN
255,Aloha Cafe,Downtown,Sweets,Closed,NaN,0,13,3,0,NaN
89,Velvet Taco,Addison,Restaurant,Open,https://www.velvettaco.com/,251,0,2,2,139.0
210,Beard Science,The Colony,Bar,Coming Soon,NaN,15,55,0,1,NaN


In [13]:
#Create new tables to load into sqldb
master_df = df[['nameID', 'neighborhoodID', 'typeID', 'statusID', 'urlID']].copy()
name_df = df[['nameID', 'Name']].copy().sort_values(by=['nameID']).drop_duplicates().reset_index(drop=True)
neighborhood_df = df[['neighborhoodID', 'Neighborhood']].copy().sort_values(by=['neighborhoodID']).drop_duplicates().reset_index(drop=True)
type_df = df[['typeID', 'Type']].copy().sort_values(by=['typeID']).drop_duplicates().reset_index(drop=True)
status_df = df[['statusID', 'Status']].copy().sort_values(by=['statusID']).drop_duplicates().reset_index(drop=True)
url_df = df[['urlID', 'URL']].copy().sort_values(by=['urlID']).drop_duplicates().reset_index(drop=True)


In [36]:
status_df.tail()

,statusID,Status
0,0,Closed
1,1,Coming Soon
2,2,Open


In [14]:
engine = create_engine('sqlite:///dallas_eats.sqlite')

In [15]:
conn = engine.connect()

In [16]:
engine.execute("DROP TABLE IF EXISTS Listings")
engine.execute("DROP TABLE IF EXISTS Name")
engine.execute("DROP TABLE IF EXISTS Neighborhood")
engine.execute("DROP TABLE IF EXISTS Type")
engine.execute("DROP TABLE IF EXISTS Status")
engine.execute("DROP TABLE IF EXISTS URL")

In [17]:
q_name = """
            CREATE TABLE Name(
                nameID INT NOT NULL PRIMARY KEY,
                Name varchar(30),
            )
"""

In [18]:
name_df.to_sql("Name", index=False, con = conn, if_exists="append")

In [19]:
q_neighborhood = """
            CREATE TABLE Neighborhood(
                NeighborhoodID INT NOT NULL PRIMARY KEY,
                Neighborhood varchar(50)
            )
"""

In [20]:
neighborhood_df.to_sql("Neighborhood", index=False, con = conn, if_exists="append")

In [21]:
q_type = """
            CREATE TABLE Type(
                typeID INT NOT NULL PRIMARY KEY,
                Type varchar(15)
            )
"""

In [22]:
type_df.to_sql("Type", index=False, con = conn, if_exists="append")

In [23]:
q_status = """
            CREATE TABLE Status(
                statusID INT NOT NULL PRIMARY KEY,
                Status varchar(15)
            )
"""

In [24]:
status_df.to_sql("Status", index=False, con = conn, if_exists="append")

In [25]:
q_url = """
            CREATE TABLE Name(
                urlID INT NOT NULL PRIMARY KEY,
                URL varchar(100)
            )
"""

In [26]:
url_df.to_sql("URL", index=False, con = conn, if_exists="append")

In [27]:
q_listings ="""
            CREATE TABLE Listings(
                id INT NOT NULL PRIMARY KEY,
                nameID INT FOREIGN KEY REFERENCES Name(nameID),
                neighborhoodID INT FOREIGN KEY REFERENCES Neighborhood(neighborhoodID),
                typeID INT FOREIGN KEY REFERENCES Type(typeID),
                statusID INT FOREIGN KEY REFERENCES Status(statusID),
                urlID INT FOREIGN KEY REFERENCES URL(urlID)
            )
"""

In [28]:
master_df.to_sql("Listings", index=False, con = conn, if_exists="append")

In [35]:
query = """
Select 
    na.Name,
    ne.Neighborhood,
    t.Type,
    s.Status,
    u.URL
FROM Listings as l
JOIN Name as na ON na.nameID = l.nameID
JOIN Neighborhood as ne ON ne.neighborhoodID = l.neighborhoodID
JOIN Type as t ON t.typeID = l.typeID
JOIN Status as s ON s.statusID = l.statusID
JOIN URL as u ON u.urlID = l.urlID
"""

engine.execute(query).fetchall()

[('AvoEatery', 'Trinity Groves', 'Restaurant', 'Open', 'https://avoeatery.com'),
 ('Bacchus', 'Grapevine', 'Restaurant', 'Open', 'https://www.hotelvin.com/dining'),
 ('Basic Taco', 'Deep Ellum', 'Restaurant', 'Open', 'http://basicta.co/'),
 ('Barcelona Wine Bar', 'Henderson', 'Restaurant', 'Open', 'https://barcelonawinebar.com/'),
 ('Bellini’s Italian Cafe', 'Design District', 'Restaurant', 'Open', 'https://www.facebook.com/BellinisItalianRestaurantBar/'),
 ('Beyond Veggies Grill', 'Plano', 'Restaurant', 'Open', 'http://www.beyondveggiesgrill.com'),
 ('Blue Cenote', 'Bishop Arts', 'Restaurant', 'Open', 'https://bluecenotedallas.com/'),
 ('Cane Rosso', 'Arlington', 'Restaurant', 'Open', 'http://www.canerosso.com'),
 ('Cava', 'Uptown', 'Restaurant', 'Open', 'https://cava.com'),
 ('Cheba Hut', 'Deep Ellum', 'Restaurant', 'Open', 'https://chebahut.com/locations/dallastx/'),
 ('Chef Chin’s Hibachi and Ramen', 'Plano', 'Restaurant', 'Open', 'http://hibachiramen.com'),
 ('Chimalma Taco Bar', 